# the Big Idea of TablArray: math operators for tables of cells

In science and engineering we often write formulas intended to mix scalar, vector, and matrix quantities.

As good programmers, we want to code our formulas to be just as happy to operate on a table of data rather than a datapoint. This means our parameters can be tables of cells, or stacks of arrays, or arrays of arrays.

Using tradition array coding, broadcasting (and a tool like einsum) can be instrumental in making this happen. But in some systems, code smells arrise especially because some convention is required to define the tabular form. How can a physics / engineering library know the application conventions, or alternately, should libraries impose such convention on applications?

TablArray is like a traditional array, with 3 new ideas that answer that call.

## Idea 1. Tabular and cellular shapes shall be distinguishable at TablArray instantiation.

Otherwise, you see a 3 dimensional array shape; is it a 1d array of matrices, a 2d array of vectors, or a 3d array of scalars?

TablArray is all about resolving this ambiguity from the beginning and upgrading math operators to match.

For example, consider a grid describing t depending on x, y:
 
| | x=0 | x=.5 | x=1 |
|---|---|---|---|
|y=-1| -15 | -15.25 | -16 |
|y=0| 5 | 4.75 | 4 |
|y=1| 25 | 24.75 | 24 |
|y=2| 45 | 44.75 | 44 |

t looks like a 4x3 array. Simple, no ambiguity here. What about a table of vectors v?

| | x=0 | x=.5 | x=1 |
|---|---|---|---|
|y=-1| [0, 1] | [.7, .7] | [1, 0] |
|y=0| [0, 1] | [.7, .7] | [1, 0] |
|y=1| [0, 1] | [.7, .7] | [1, 0] |
|y=2| [0, 1] | [.7, .7] | [1, 0] |

And, to make things more interesting, also note that is degenerate along y, so let's simplify. Then v is a 3x2 array.

| x=0 | x=.5 | x=1 |
|---|---|---|
| [0, 1] | [.7, .7] | [1, 0] |

But now we can't write a simple function like <code>t * v</code> anymore. The shape (3, 2) is not broadcast-compatible with (3, 4), so <code>t * v</code> will return an error (as it should). But you can see what we want from <code>t * v</code>.

How do we define "shape" to distinguish between tabular and cellular shape? The shape of t could be described like (3, 4 |), then the shape of v would be (3, 1 | 2). '|' is a new symbol to mark where tabular shape ends and cellular shape begins.

TablArray's benefits will derive from this basic idea.

## Idea 2. Broadcasting rules then consider tabular **and** cellular shapes.

The first, and most powerful, benefit is an upgrade to broadcasting rules. In other words, tables are broadcast, and at the same time, cells are broadcast.

In the above example, the degenerate table shapes (3) and (4, 3) will be broadcast together. And at the same time the scalar cells shapes () and (2) will be broadcast.

### Code example

Let's code the <code>t * v</code> example from earlier.

In [1]:
import numpy as np
import tablarray as ta
x = ta.TablArray(np.linspace(0, 1, 3), 0)
# notice 0 after all the linspace()
# this tells TablArray that cdim=0, meaning the cells are 0 dim
# TablArray always needs to know cdim
y = ta.TablArray(np.linspace(-1, 2, 4).reshape(4, 1), 0)
# again cdim=0, so cells are scalars
t = 20*y + 5 - 1 * x**2
v = ta.TablArray([[1, 0], [.7, .7], [0, 1]], 1)
# this time cdim=1, so cells are vectors

In [2]:
print(x)
print(y)

[|0.  0.5 1. |]t(3,)|c()
[[|-1.|]
 [| 0.|]
 [| 1.|]
 [| 2.|]]t(4, 1)|c()


In [3]:
print(t)
print(v)

[[|-15.   -15.25 -16.  |]
 [|  5.     4.75   4.  |]
 [| 25.    24.75  24.  |]
 [| 45.    44.75  44.  |]]t(4, 3)|c()
[|[1.  0. ]|
 |[0.7 0.7]|
 |[0.  1. ]|]t(3,)|c(2,)


In [4]:
print(t * v)

[[|[-15.     -0.   ]|
  |[-10.675 -10.675]|
  |[ -0.    -16.   ]|]

 [|[  5.      0.   ]|
  |[  3.325   3.325]|
  |[  0.      4.   ]|]

 [|[ 25.      0.   ]|
  |[ 17.325  17.325]|
  |[  0.     24.   ]|]

 [|[ 45.      0.   ]|
  |[ 31.325  31.325]|
  |[  0.     44.   ]|]]t(4, 3)|c(2,)


Notice that when we print TablArray parameters, '|' is used to show the separation between tabular and cellular structure. Also there is a full shape descriptor like 't(4, 3)|c(2,)' at the tail of the string.

### Alternatives

Nothing that TablArray can do is 100% unique.

Do you want broadcasting to always work like the <code>t * v</code> example? There is a convention you can use that accomplishes the same thing with arrays of arrays. When you start coding your system, decide what the maximum cell dimension is, probably 1 or 2 (vectors or matrices). Once this decision is made you need to be consistent throughout all your code. Whenever you instantiate a parameter, pad the cells shape on the left with 2s up to the max cell dimension.

For the working example above, the shape of t would need to be (4, 3, 1), while the shape of v would be (4, 1, 1). Here's how that would look in tradition code:

In [5]:
x_olde = np.linspace(0, 1, 3).reshape(3, 1)
y_olde = np.linspace(-1, 2, 4).reshape(4, 1, 1)
t_olde = 20*y_olde + 5 - 1 * x_olde**2
v_olde = np.array([[1, 0], [.7, .7], [0, 1]])
print(t_olde * v_olde)

[[[-15.     -0.   ]
  [-10.675 -10.675]
  [ -0.    -16.   ]]

 [[  5.      0.   ]
  [  3.325   3.325]
  [  0.      4.   ]]

 [[ 25.      0.   ]
  [ 17.325  17.325]
  [  0.     24.   ]]

 [[ 45.      0.   ]
  [ 31.325  31.325]
  [  0.     44.   ]]]


That's it for broadcasting. Now for the final basic principle of TablArray's:

## Idea 3. Matrix operations like matmul always align to cells.

In traditional stacks of arrays, you would need to use einsum to implement matmul safely. In fact, tablarray uses numpy.einsum under the hood for matmul. You may have already known about this, and it may not seem like a big deal to do it yourself, even though some engineers will find einsum pretty hard to read. But more often, engineers don't know this and spend time writing outer loops to descend into their stacks and execute matmul on the inner arrays. That's executionally inefficient, as well as inflexible toward varying tabular shape. Experienced coders know to avoid loops and write vectorized code instead.

TablArray's table/cell distinction is needed to make matmul always unambiguous for any pair of compatible parameters. Ultimately what this delivers to all engineers is the power of a name. If what you mean is matmul (or the binary operator '@') - call it what it is.

## Conclusions

I would suggest readers first consider the shape of their parameters, and variability in those shapes, to see whether they have a use case for TablArray.

TablArray is not unique but may be more readable and/or bug resistant by way of being unambiguous and natural. In other words, traditional stacks of arrays may be abused as often as well used.

I would urge the reader to also consider the next levels. TablArray enables more big ideas. Head back to the index to learn about:

1. TablaSet, a key-indexed dataset of mutually broadcast-compatible parameters. It bears resemblance to a database, but with fast advanced slicing called projection.
2. TablaSolve, an object oriented dataset where user-defined solver methods are attached. Anytime seed data is presented or manipulated, solvers can be automatically arranged to determine downstream parameters.

## More info: indexing

TablArray objects have a property .view which determines the behavior of indexing. Views make use of pass-by-reference. For example if t1 is a TablArray then

* t1.cell[0] will return a view of t1's table of all the 0th elements of the cells.
* t1.table[0] will return a view of t1's 0th cell.
* t1.bcast is like table but uses projection - there is a better discussion under TablaSet. Technically what this means is that if you index missing dimensions or dimensions which have 1 length, you will get an answer instead of an error. This is like using broadcasting rules during indexing. But the term 'projection' makes more tactile sense: if you call for a row of a 2d table, you get a row, while if you call for a row of a 1d table, you get a scalar. That's not the same as indexing; where if you call for a row of a 1d table you get an error.
* t1.array is a view of t1 which will ignore tabular and cellular distinction. E.g. a shape like t(4, 3)|c(1) will be indexed the same as a traditionally (4, 3, 1) array.

Example code:

In [6]:
print(v.table[1])

[0.7 0.7]


In [7]:
print(v.cell[1])

[|0.  0.7 1. |]t(3,)|c()


TablArray objects also have a .base property which is actually the numpy.ndarray data that implements the TablArray object. In case you find the need to mix TablArray and other ndarray methods.